In [3]:
##Import necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
from IPython.display import clear_output
##Import bq_helper to pull forestry data from Kaggle
from bq_helper import BigQueryHelper #pip install -e git+https://github.com/SohierDane/BigQuery_Helper#egg=bq_helper
import os
#Setup Google Credentials(Data is stored on Google Cloud Servers)
json = '/home/andrew/Desktop/ErdosBootCamp/GroupProject/erdosproject-c9b945dbfac5.json'

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= json #Get .json file from Google Cloud Platform

In [4]:
##Connect to USFS FIA data and list the available datasets
usfs = BigQueryHelper("bigquery-public-data", "usfs_fia")
#usfs.list_tables()

DefaultCredentialsError: File /home/andrew/Desktop/ErdosBootCamp/GroupProject/erdosproject-c9b945dbfac5.json was not found.

In [3]:
query1 = f"""
    SELECT
        c.plot_sequence_number,
        c.physiographic_class_code,
        c.growingstock_stocking_code,
        c.owner_class_code,
        p.plot_state_code,
        p.plot_county_code
    FROM 
      `bigquery-public-data.usfs_fia.condition`  c
    JOIN 
      `bigquery-public-data.usfs_fia.plot`  p
        ON p.plot_sequence_number = c.plot_sequence_number
    ;        """
df = usfs.query_to_pandas_safe(query1, max_gb_scanned=10)

In [4]:
df_edits = df.drop(df.index[df.plot_state_code > 56])

In [5]:
state_codes = df_edits.plot_state_code.values
county_codes = df_edits.plot_county_code.values

df_edits['state_county_code'] = [f'{state:03}{county:03}' for state,county in zip(state_codes, county_codes)]

NameError: name 'df_edits' is not defined

In [6]:
df_edits.physiographic_class_code.loc[df_edits.physiographic_class_code.isna()] = 0

/home/andrew/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [7]:
df_edits.growingstock_stocking_code.loc[df_edits.growingstock_stocking_code == ''] = str(0)

In [8]:
df_edits['physio_class_code'] = (df_edits.physiographic_class_code - df_edits.physiographic_class_code % 10) / 10

In [9]:
df_edits['growing_stocking_code'] = df_edits.growingstock_stocking_code.values.astype(np.float)

In [10]:
df_edits.sample(10)

,plot_sequence_number,physiographic_class_code,growingstock_stocking_code,owner_class_code,plot_state_code,plot_county_code,state_county_code,physio_class_code,growing_stocking_code
2076739,10734298010661,0.0,0,46.0,42,49,042049,0.0,0.0
541123,289514970489998,0.0,0,NaN,12,91,012091,0.0,0.0
2301167,163377548010854,0.0,4,25.0,47,105,047105,0.0,4.0
1301729,102143860010661,0.0,0,NaN,27,161,027161,0.0,0.0
853286,3448974010661,0.0,0,NaN,19,3,019003,0.0,0.0
2023571,238861459489998,23.0,1,11.0,41,15,041015,2.0,1.0
62435,144994503010661,0.0,0,NaN,25,27,025027,0.0,0.0
2086652,306268363489998,0.0,0,NaN,42,11,042011,0.0,0.0
2725919,249881125489998,0.0,0,NaN,55,57,055057,0.0,0.0
1813217,161222548010854,22.0,3,32.0,37,109,037109,2.0,3.0


In [14]:
state_county = df_edits.state_county_code.unique()
physiographic_class_code = []
growingstock_stocking_code = []

for elem in state_county:
    psg = df_edits.loc[df_edits.state_county_code == elem].physio_class_code.value_counts().index
    if len(psg) == 1:
        physiographic_class_code.append(psg[0])
    elif psg[0] == 0:
        physiographic_class_code.append(psg[1])
    else:
        physiographic_class_code.append(psg[0])
        
    gss = df_edits.loc[df_edits.state_county_code == elem].growing_stocking_code.value_counts().index
    if len(gss) == 1:
        growingstock_stocking_code.append(gss[0])
    elif gss[0] == 0:
        growingstock_stocking_code.append(gss[1])
    else:
        growingstock_stocking_code.append(gss[0])
        


In [17]:
county_df = pd.DataFrame({'state_county' : state_county,
                         'physiographic_class_code' : physiographic_class_code,
                         'growingstock_stocking_code' : growingstock_stocking_code})

In [34]:
county_df['state_code'] = [int(elem[:3]) for elem in county_df['state_county'].values]

In [35]:
county_df['county_code'] = [int(elem[3:]) for elem in county_df['state_county'].values]

In [36]:
county_df

,state_county,physiographic_class_code,growingstock_stocking_code,state_code,county_code
0,009009,2.0,2.0,9,9
1,009005,2.0,2.0,9,5
2,009011,2.0,3.0,9,11
3,009013,2.0,2.0,9,13
4,024045,2.0,2.0,24,45
...,...,...,...,...,...
3086,056043,1.0,5.0,56,43
3087,056017,1.0,5.0,56,17
3088,056005,1.0,5.0,56,5
3089,056031,1.0,5.0,56,31


In [37]:
county_df.to_csv('categoricals.csv')